# Training Temporal Convolutional Neural Network on Landsat Satellite Image Time Series to Detect Leafy Spurge

This Notebook aims to develop a temporal convolutional neural network based on a time series of Landsat satellite imagery and class labels from the National Land Cover Dataset (NLCD). 

Table of Contents
-
-
-
-
-
-
-
-


Contributors:
Thomas Lake

Version: November 2022


# Imports

In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import pprint
import time
import math
import random
import glob
from functools import reduce
from pprint import pprint


#Plotting
import matplotlib.pyplot as plt
from IPython.display import clear_output


# Tensorflow Setup

In [2]:
# Tensorflow setup.

# Tensorflow version 2.4.1
import tensorflow as tf
print(tf.__version__) 

# Keras setup.
import keras
from keras import layers
from keras.layers import Flatten
from keras import backend as K
from keras import regularizers
from keras import optimizers
from keras.regularizers import l2
from keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, Flatten, Lambda, SpatialDropout1D, Concatenate
from keras.layers import Conv1D, Conv2D, AveragePooling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.callbacks import Callback, ModelCheckpoint, History, EarlyStopping
from keras.models import Model, load_model
from keras.utils.np_utils import to_categorical
from keras import backend as K


2022-11-21 11:28:18.443354: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/moeller/lakex055/.conda/envs/tf_gpu/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.4.1


In [ ]:
# Are We Using a GPU?

import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))
# [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

print(tf.test.is_built_with_cuda)
# <function is_built_with_cuda at 0x7f4f5730fbf8>

print(tf.test.gpu_device_name())
# /device:GPU:0

print(tf.config.get_visible_devices())
# [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Matrix multiplication test with gpu

a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)

# Import temporalCNN Python modules
# https://github.com/charlotte-pel/temporalCNN/

In [ ]:
import sys

# Import from ~/sits folder
# Contains readingsits.py file to read and compute spectral features on SITS
sys.path.append("/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/temporalCNN/sits")
import readingsits

# Import from ~/deeplearning folder
# Contains multiple .py files with varying DL architectures 
sys.path.append("/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/temporalCNN/deeplearning")

import architecture_features
import architecture_complexity
import architecture_rnn
import architecture_regul
import architecture_batchsize
import architecture_depth
import architecture_spectro_temporal
import architecture_pooling

# Import from ~/outputfiles folder
# Contains evaluation.py and save.py files with fucntions to compute summary statistics, write predictions, and create confusion matrices
sys.path.append("/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/temporalCNN/outputfiles")

import evaluation
import save

Define Variables



Read in Full Training/Testing/Validation Dataset sampled from 1M points & NLCD classes


In [3]:
# Read data into pandas df from .csv
df = pd.DataFrame()

# Dataframe contains N rows, 67 columns
df = pd.read_csv("/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/datasets_oct22/NLCD2001_full_dataset_oct2022.csv", header='infer')
print(df.shape)

list(df.columns)

(727297, 67)


['Unnamed: 0',
 'X0_BlueMarchApril2000',
 'X0_GreenMarchApril2000',
 'X0_RedMarchApril2000',
 'X0_NIRMarchApril2000',
 'X0_SWIR1MarchApril2000',
 'X0_SWIR2MarchApril2000',
 'X0_NDVIMarchApril2000',
 'X0_BlueMayJune2000',
 'X0_GreenMayJune2000',
 'X0_RedMayJune2000',
 'X0_NIRMayJune2000',
 'X0_SWIR1MayJune2000',
 'X0_SWIR2MayJune2000',
 'X0_NDVIMayJune2000',
 'X0_BlueJulyAug2000',
 'X0_GreenJulyAug2000',
 'X0_RedJulyAug2000',
 'X0_NIRJulyAug2000',
 'X0_SWIR1JulyAug2000',
 'X0_SWIR2JulyAug2000',
 'X0_NDVIJulyAug2000',
 'X1_BlueMarchApril2001',
 'X1_GreenMarchApril2001',
 'X1_RedMarchApril2001',
 'X1_NIRMarchApril2001',
 'X1_SWIR1MarchApril2001',
 'X1_SWIR2MarchApril2001',
 'X1_NDVIMarchApril2001',
 'X1_BlueMayJune2001',
 'X1_GreenMayJune2001',
 'X1_RedMayJune2001',
 'X1_NIRMayJune2001',
 'X1_SWIR1MayJune2001',
 'X1_SWIR2MayJune2001',
 'X1_NDVIMayJune2001',
 'X1_BlueJulyAug2001',
 'X1_GreenJulyAug2001',
 'X1_RedJulyAug2001',
 'X1_NIRJulyAug2001',
 'X1_SWIR1JulyAug2001',
 'X1_SWIR2JulyAug2

In [ ]:
# Number of class instances in the 2001 full dataset
df['class'].value_counts()


Format the data frame for use with the temporalCNN modules. Sample dataset format found in temporalCNN/example folder

1. Drop the first column ('Unnamed: 0') and the latitude/longitude columns.
2. Move the class column to the first index
3. Add an index column to the second position (a unique ID)

In [ ]:
# Remove the first column ("Unnamed:0"), and latitude/longitude columns
df.drop(['Unnamed: 0', 'lat', 'lon'], axis=1, inplace=True)

# Move class column ("class") to first index
cls_column = df.pop("class")
df.insert(0, "class", cls_column)

# Set the class column as the index
df.set_index('class', inplace=True)

# Add column in second position as an unique ID ("index")
df.insert(0, "index", range(1, 1 + len(df)))

df.head(5)


# Divide full dataset into training and testing subsets, export to csv

Use Scikit Learn train test split: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Quick utility that wraps input validation and next(ShuffleSplit().split(X, y)) and application to input data into a single call for splitting (and optionally subsampling) data in a oneliner.

In [ ]:
#from sklearn.model_selection import train_test_split

#train_df, test_df = train_test_split(df, test_size=0.1)

#Number of rows/columns in dataset
#print(train_df.shape, test_df.shape)

# Write full training and testing dataframes to CSV
# Export format as rows [classID, index, band_values/timeseries...] with no header
# train_df.to_csv("/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/datasets_oct22/train_dataset_2001_full_oct2022.csv", header=False)
# test_df.to_csv("/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/datasets_oct22/test_dataset_2001_full_oct2022.csv", header=False)


In [ ]:
# Useful Block - Keep this!
# Combine multiple CSV files with identical columns into one CSV & export

#from pathlib import Path
#import pandas as pd
#import numpy as np

#path = r'/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/datasets_oct22' # use your path

#all_files = glob.glob(os.path.join(path, "train_*.csv"))

#df = pd.concat((pd.read_csv(f, header=None) for f in all_files), ignore_index=True)

#df.to_csv('/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/datasets_oct22/train_dataset_allyears_full_oct22.csv', sep=',', header=False, index=False)

# Set Model Paths for Input Data & Model Results Files

In [ ]:
# Set the path to exported training/testing dataset
sits_path = '/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/datasets_oct22'    
    
# Set Architecture / Model Run Index (used if running in batch on MSI)
noarchi = 0
norun = 0
feature = "SB" #use only spectral bands provided (do not compute new bands, like NDVI, which are already computed)

# Parameters to set
n_channels = 7 #-- B G NDVI NIR Red SWIR1 SWIR2
val_rate = 0.1 # Validation data rate

# String variables for the training and testing datasets
train_str = 'train_dataset_allyears_full_oct22'
test_str = 'test_dataset_allyears_full_oct22'					

# Get filenames
train_file = sits_path + '/' + train_str + '.csv'
test_file = sits_path + '/' + test_str + '.csv'
print("train_file: ", train_file)
print("test_file: ", test_file)


In [ ]:

# Set a model results path
res_path = '/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/temporalCNN'

# Creating output path if does not exist
if not os.path.exists(res_path):
  print("ResPath DNE")
  os.makedirs(res_path)

# Evaluated metrics
eval_label = ['OA', 'train_loss', 'train_time', 'test_time']	
	

# Output files			
res_path = res_path + '/Archi' + str(noarchi) + '/'
if not os.path.exists(res_path):
  os.makedirs(res_path)
  print("noarchi: ", noarchi)

# Create output files to capture model results
str_result = feature + '-' + train_str + '-noarchi' + str(noarchi) + '-norun' + str(norun) 
res_file = res_path + '/resultOA-' + str_result + '.csv'
res_mat = np.zeros((len(eval_label),1))
traintest_loss_file = res_path + '/trainingHistory-' + str_result + '.csv'
conf_file = res_path + '/confMatrix-' + str_result + '.csv'
out_model_file = res_path + '/bestmodel-' + str_result + '.h5'





# Read, Reshape, and Normalize Training Dataset for Temporal CNN
# https://github.com/charlotte-pel/temporalCNN/blob/master/run_archi.py

The readSITSData function is located in temporalCNN/sits/readingsits.py 
The function outputs X (variable vectors for each sample/row), polygon_ids (unique id for each row), and Y (land class label for each sample/row)

I am not sure if normalizing the data here will cause issues when predicting on (non-normalized) Landsat imagery.

In [ ]:
from tensorflow.keras.utils import to_categorical

# Read in SITS training and testing datasets
X_train, polygon_ids_train, y_train = readingsits.readSITSData(train_file)
X_test,  polygon_ids_test, y_test = readingsits.readSITSData(test_file)
print(X_test)  #verify spectral band data looks correct
print(X_test.shape) #num_samples, 63 bands (9 timesteps * 7 bands/timestep = 63)


# Number of unique classes in y_train and y_test datasets should = 9
n_classes_test = len(np.unique(y_test))
print(n_classes_test)
n_classes_train = len(np.unique(y_train))
print(n_classes_train)

# heck equal number of classes in training and testing dataset
if(n_classes_test != n_classes_train):
  print("WARNING: different number of classes in train and test")

n_classes = max(n_classes_train, n_classes_test) # 9 classes
y_train_one_hot = to_categorical(y_train) # specify number of classes explicity - may need to recode classes sequentially (1-9) to work correctly?
y_test_one_hot = to_categorical(y_test)

print(y_test_one_hot) #verify one hot encoding was successful
print(y_test_one_hot.shape)
print(y_test_one_hot[0])





# Reshape data

In [ ]:
X_train = readingsits.addingfeat_reshape_data(X_train, feature, n_channels) #Feature = "SB" (spectral bands)

X_test = readingsits.addingfeat_reshape_data(X_test, feature, n_channels)		

print(X_train[0, :, :])
print(X_train.shape) #verify reshape was successful, now num_samples, num_timesteps, num_bands


# Create the Validation Data Set

In [ ]:
#---- Extracting a validation set (if necesary)
if val_rate > 0:
  #Number of samples to take from Training dataset based on validation rate
  val_num_samples = int(math.ceil(X_train.shape[0] * val_rate))

  #Select random indices for val_num_samples to select validation set
  val_indices = random.sample(range(1, X_train.shape[0]), val_num_samples)
  #remove these indices from the training set
  train_indices = np.delete(range(1, X_train.shape[0]), val_indices)

  #Create training and validation sets 
  X_val = X_train[val_indices, :]
  y_val = y_train[val_indices]
  X_train = X_train[train_indices, :]
  y_train = y_train[train_indices]

  #--- Computing the one-hot encoding (recomputing it for train)
  y_train_one_hot = to_categorical(y_train)
  y_val_one_hot = to_categorical(y_val)

  n_classes_val = len(np.unique(y_val))
  print(n_classes_val)
  n_classes_train = len(np.unique(y_train))
  print(n_classes_train)

  #Check equal number of classes in training and testing dataset
  if(n_classes_val != n_classes_train):
    print("WARNING: different number of classes in train and test")
  

print(X_train.shape, y_train_one_hot.shape, X_val.shape, y_val_one_hot.shape, X_test.shape, y_test_one_hot.shape)

In [ ]:
#-----------------------------------------------------------------------		
def conv_bn(X, **conv_params):	
	nbunits = conv_params["nbunits"];
	kernel_size = conv_params["kernel_size"];

	strides = conv_params.setdefault("strides", 1)
	padding = conv_params.setdefault("padding", "same")
	kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-6))
	kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")

	Z = Conv1D(nbunits, kernel_size=kernel_size, 
			strides = strides, padding=padding,
			kernel_initializer=kernel_initializer,
			kernel_regularizer=kernel_regularizer)(X)

	return BatchNormalization(axis=-1)(Z) #-- CHANNEL_AXIS (-1)

#-----------------------------------------------------------------------		
def conv_bn_relu(X, **conv_params):
	Znorm = conv_bn(X, **conv_params)
	return Activation('relu')(Znorm)
	
#-----------------------------------------------------------------------		
def conv_bn_relu_drop(X, **conv_params):	
	dropout_rate = conv_params.setdefault("dropout_rate", 0.5)
	A = conv_bn_relu(X, **conv_params)
	return Dropout(dropout_rate)(A)

#-----------------------------------------------------------------------		
def fc_bn(X, **fc_params):
	nbunits = fc_params["nbunits"];
	
	kernel_regularizer = fc_params.setdefault("kernel_regularizer", l2(1.e-6))
	kernel_initializer = fc_params.setdefault("kernel_initializer", "he_normal")
		
	Z = Dense(nbunits, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)(X)
	return BatchNormalization(axis=-1)(Z) #-- CHANNEL_AXIS (-1)
	
#-----------------------------------------------------------------------		
def fc_bn_relu(X, **fc_params):	
	Znorm = fc_bn(X, **fc_params)
	return Activation('relu')(Znorm)

#-----------------------------------------------------------------------		
def fc_bn_relu_drop(X, **fc_params):
	dropout_rate = fc_params.setdefault("dropout_rate", 0.5)
	A = fc_bn_relu(X, **fc_params)
	return Dropout(dropout_rate)(A)

#-----------------------------------------------------------------------		
def softmax(X, nbclasses, **params):
	kernel_regularizer = params.setdefault("kernel_regularizer", l2(1.e-6))
	kernel_initializer = params.setdefault("kernel_initializer", "glorot_uniform")
	return Dense(nbclasses, activation='softmax', 
			kernel_initializer=kernel_initializer,
			kernel_regularizer=kernel_regularizer)(X)


# Create the Deep Learning Model in Keras

Before we create the model, there's still a wee bit of pre-processing to get the data into the right input shape and a format that can be used with cross-entropy loss. Specifically, Keras expects a list of inputs and a one-hot vector for the class. (See the Keras loss function docs, the TensorFlow categorical identity docs and the tf.one_hot docs for details).

Here we will use a simple neural network model with a 64 node hidden layer, a dropout layer and an output layer. Once the dataset has been prepared, define the model, compile it, fit it to the training data. See the Keras Sequential model guide for more details.

In [ ]:

#Custom Model Architecture inspired by Allred at al. 2021

#Get input sizes
m, L, depth = X_train.shape
input_shape = (L, depth)

#-- parameters of the architecture
nbclasses = 10
l2_rate = 1.e-6
dropout_rate = 0.1
nb_conv = 3
nb_fc= 1
nbunits_conv = 64 #-- will be double
nbunits_fc = 256 #-- will be double

	# Define the input placeholder.
X_input = Input(input_shape)
		
	#-- nb_conv CONV layers
X = X_input
for add in range(nb_conv):
    X = conv_bn_relu_drop(X, nbunits=nbunits_conv, kernel_size=5, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
	#-- Flatten + 	1 FC layers
X = Flatten()(X)
for add in range(nb_fc):	
    X = fc_bn_relu_drop(X, nbunits=nbunits_fc, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
		
	#-- SOFTMAX layer
out = softmax(X, nbclasses, kernel_regularizer=l2(l2_rate))
		
	# Create model.
model = Model(inputs = X_input, outputs = out, name='Archi_3CONV64_1FC256')	



In [ ]:
###
# Define Model Variables
###

# Model variables
n_epochs = 1000
batch_size = 2048
#lr = 0.0001 #recommended in Allred et al., 2021
#beta_1 = 0.9 #not used, but can be used to modify optimizer LR
#beta_2 = 0.999
#epsilon = 1e-07

#Define Class Weights
from sklearn.utils import class_weight

class_weights =  class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train)

class_weights = dict(zip(np.unique(y_train), class_weights))

# Class weights function
# squared of inverse frequenecy weights?
# inverse of frequency
class_weights = {0: 0,
                 1: 7.046028630719989,
                 2: 3.6421837069230087,
                 3: 31.37461158722999,
                 4: 0.7614511317372198,
                 5: 0.6015453322153169,
                 6: 0.3652990948014909,
                 7: 0.39487324200412083,
                 8: 4.334510403657227,
                 9: 20.275284755853498}

print(class_weights)



# Model callbacks

In [ ]:



checkpoint = ModelCheckpoint(out_model_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10, verbose=1, mode='auto', restore_best_weights=False)

#Plot Loss and Accuracy Callback
class PlotLearning(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.f1 = []
        self.val_f1 = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.f1.append(logs.get('accuracy'))
        self.val_f1.append(logs.get('val_accuracy'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val loss")
        ax1.legend()
        
        ax2.plot(self.x, self.f1, label="Acc")
        ax2.plot(self.x, self.val_f1, label="val Acc ")
        ax2.legend()
        
        plt.show();
        
plot_losses = PlotLearning()


# Learning Rate Warmup and Decay Callback
def lr_warmup_cosine_decay(global_step,
                           warmup_steps,
                           hold = 0,
                           total_steps=0,
                           start_lr=0.0,
                           target_lr=1e-3):
    # Cosine decay
    learning_rate = 0.5 * target_lr * (1 + np.cos(np.pi * (global_step - warmup_steps - hold) / float(total_steps - warmup_steps - hold)))

    # Target LR * progress of warmup (=1 at the final warmup step)
    warmup_lr = target_lr * (global_step / warmup_steps)

    # Choose between `warmup_lr`, `target_lr` and `learning_rate` based on whether `global_step < warmup_steps` and we're still holding.
    # i.e. warm up if we're still warming up and use cosine decayed lr otherwise
    if hold > 0:
        learning_rate = np.where(global_step > warmup_steps + hold,
                                 learning_rate, target_lr)
    
    learning_rate = np.where(global_step < warmup_steps, warmup_lr, learning_rate)
    return learning_rate

#Plot the learning rate schedule
steps = np.arange(0, 1000, 1)
lrs = []

for step in steps:
  lrs.append(lr_warmup_cosine_decay(step, total_steps=len(steps), warmup_steps=50, hold=5))
plt.plot(lrs)


class WarmupCosineDecay(keras.callbacks.Callback):
    def __init__(self, total_steps=0, warmup_steps=0, start_lr=0.0, target_lr=1e-3, hold=0):

        super(WarmupCosineDecay, self).__init__()
        self.start_lr = start_lr
        self.hold = hold
        self.total_steps = total_steps
        self.global_step = 0
        self.target_lr = target_lr
        self.warmup_steps = warmup_steps
        self.lrs = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = model.optimizer.lr.numpy()
        self.lrs.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = lr_warmup_cosine_decay(global_step=self.global_step,
                                    total_steps=self.total_steps,
                                    warmup_steps=self.warmup_steps,
                                    start_lr=self.start_lr,
                                    target_lr=self.target_lr,
                                    hold=self.hold)
        K.set_value(self.model.optimizer.lr, lr)
        
        
# If already batched
# If not batched
total_steps = n_epochs
# 5% of the steps
warmup_steps = int(0.05*total_steps)

warmup_callback = WarmupCosineDecay(total_steps=total_steps, 
                             warmup_steps=warmup_steps,
                             hold=int(warmup_steps/2), 
                             start_lr=0.0, 
                             target_lr=1e-3)



In [ ]:

#Model Optimizer
#opt = tf.keras.optimizers.Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon)

# Compile Model
model.compile(optimizer = 'adam', loss = "mean_squared_error", metrics = ["accuracy"])

model.summary()

#Train the model
start_train_time = time.time()

# Fit the model
hist = model.fit(x = X_train, 
                 y = y_train_one_hot, 
                 epochs = n_epochs,
                 batch_size = batch_size, 
                 shuffle=True,
                 validation_data=(X_val, y_val_one_hot), 
                 verbose=1,
                 callbacks = [plot_losses, warmup_callback])


train_time = round(time.time()-start_train_time, 2)


In [ ]:
# Evaluate the model prediction

from sklearn.metrics import multilabel_confusion_matrix
from tabulate import tabulate

# Predict the model on withheld testing dataset
y_pred = model.predict(X_test)

y_pred = np.argmax(y_pred, axis=-1)
y_pred_flat = y_pred.flatten()
y_pred_flat = y_pred_flat.astype(int)

y_test = y_test.astype(int)    
y_test_flat = y_test.flatten()


# Calculate confusion matrix
class_names = ["Water", "Developed", "BarrenLand", "Forest", "Shrub/Scrub", "Grassland/Herbaceous", "Croplands", "EmergentWetlands", "LeafySpurge"]
class_labels = [1, 2, 3, 4, 5, 6, 7, 8, 9]
c = multilabel_confusion_matrix(y_test_flat, y_pred_flat, labels = class_labels)
model_output_metrics = []
for i in range(len(class_labels)):
    tn=c[i, 0, 0]
    tp=c[i, 1, 1]
    fn=c[i, 1, 0]
    fp=c[i, 0, 1]
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    TPR_Sens_Recall = tp/(tp+fn)
    TNR_Spec = tn/(tn+fp)
    FPR = fp/(fp+tn)
    FNR = fn/(fn+tp)
    precision = tp/(tp+fp)
    jaccard = tp/(tp+fp+fn)
    beta = 0.5
    F05 = ((1 + beta**2) * precision * TPR_Sens_Recall) / (beta**2 * precision + TPR_Sens_Recall)
    beta = 1
    F1 = ((1 + beta**2) * precision * TPR_Sens_Recall) / (beta**2 * precision + TPR_Sens_Recall)
    beta = 2
    F2 = ((1 + beta**2) * precision * TPR_Sens_Recall) / (beta**2 * precision + TPR_Sens_Recall)
    outputs = [class_names[i], tp, tn, fp, fn, accuracy, TPR_Sens_Recall, TNR_Spec, FPR, FNR, precision, jaccard, F1]
    model_output_metrics.append(outputs)

# Print and format outputs
print(tabulate(model_output_metrics, floatfmt=".2f", headers=["Class Name", "TP", "TN", "FP", "FN", "Accuracy", "TPR/Sens/Recall", "TNR/Spec", "FPR", "FNR", "Precision", "Jaccard", "F1"]))





In [ ]:

import os
import sys
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.io import imread, imshow, imsave
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix, cohen_kappa_score, accuracy_score, f1_score, precision_score, recall_score, jaccard_score, fbeta_score
from tensorflow.keras.models import load_model
from tabulate import tabulate

def plot_confusion_matrix(
        y_true,
        y_pred,
        classes,
        test_name,
        normalize=False,
        set_title=False,
        save_fig=False,
        cmap=plt.cm.Blues
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    if set_title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    # and save it to log file
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
        #with open(f'F:/PlanetScope_LSTM_Imagery/reports/logs_and_plots/{test_name}_log.txt', 'ab') as f:
        #    f.write(b'\nNormalized confusion matrix\n')
        #    np.savetxt(f, cm, fmt='%.3f')
    else:
        print('Confusion matrix, without normalization')
        #with open(f'F:/PlanetScope_LSTM_Imagery/reports/logs_and_plots/{test_name}_log.txt', 'ab') as f:
        #    f.write(b'\nConfusion matrix, without normalization\n')
        #    np.savetxt(f, cm, fmt='%7u')

    print(cm)
    #cm = cm[1:10]
    #cm = cm[:,1:]

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    if normalize:
        im.set_clim(0., 1.)     # fixes missing '1.0' tick at top of colorbar
    cb = ax.figure.colorbar(im, ax=ax)
    if normalize:
        cb.set_ticks(np.arange(0., 1.2, 0.2))
        cb.set_ticklabels([f'{i/5:.1f}' for i in range(6)])
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title if set_title else None,
           ylabel='True label',
           xlabel='Predicted label')
    ax.set_ylim(len(cm)-0.5, -0.5)
    ax.xaxis.label.set_size(10)
    ax.yaxis.label.set_size(10)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            if np.round(cm[i, j], 2) > 0.:
                ax.text(j, i, format(cm[i, j], fmt),
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")
            else:
                ax.text(j, i, '–',
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

    if save_fig:
        if normalize:
            plt.savefig(f'F:/PlanetScope_LSTM_Imagery/reports/logs_and_plots/{test_name}_cm_normal.pdf')
        else:
            plt.savefig(f'F:/PlanetScope_LSTM_Imagery/reports/logs_and_plots/{test_name}_cm_non_normal.pdf')
    return fig, ax




inv_category_dict = {1:"Water", 2:"Developed", 3:"BarrenLand", 4:"Forest", 5:"Shrub/Scrub", 6:"Grassland/Herbaceous", 7:"Croplands", 8:"EmergentWetlands", 9:"LeafySpurge"}

class_names = [inv_category_dict[i] for i in np.arange(1, 10)]


# save plot of normalized cm
plot_confusion_matrix(
    y_test_flat,
    y_pred_flat,
    classes=class_names,
    test_name="myModel",
    normalize=True,
    save_fig=False
)



In [ ]:
# Calculate RMSE


# Predict the model on withheld testing dataset
y_pred_probs = model.predict(X_test)


def computingRMSE(y_test_one_hot,p_test):
	""""
		Computing RMSE from the prediction of the softmax layer
		INPUT:
			- y_test_one_one: one hot encoding of the test labels
			- p_test: predicted 'probabilities' from the model for the test instances
		OUTPUT:
			- rmse: Root Mean Square Error
	"""
	nbTestInstances = y_test_one_hot.shape[0]
	diff_proba = y_test_one_hot - p_test
	return math.sqrt(np.sum(diff_proba*diff_proba)/nbTestInstances)

print(computingRMSE(y_test_one_hot, y_pred_probs))



In [ ]:

# Save the Trained Model as a .h5 file
model.save(r'/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/temporalCNN/Archi0/secondmodel.h5')
           
           

In [ ]:

# Load a trained model
model = keras.models.load_model(r'/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/temporalCNN/Archi0/draft_cnn_modeL_300epochs_oct272022.h5')


In [ ]:

#Model Prediction on small TIF raster file

import rasterio
from rasterio.plot import show
import glob

# Not used, but might be needed to index across tiles
tile_index = 170

# Input prediction .tif path
image_path = r'/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/datasets_oct22/rasters_gcloud/'

# Output prediction file path
outpath = r'/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/datasets_oct22/raster_predictions_gcloud/'

# List all .tif files in /rasters folder for prediction
tif_image_list = glob.glob(image_path + '*.tif')

print(tif_image_list[0:2])

# Loop through every tif file for prediction.
for t in range(len(tif_image_list)):
    
    # Open .tif array image with rasterio, read to numpy array
    with rasterio.open(tif_image_list[t], 'r') as ds:
        arr = ds.read()  # read all raster values

    # Define shape of input .tif image
    bands, width, height = arr.shape
    
    # Convert Data Type to float32 by division.
    arr = arr/10000
    
    # Reshape .tif array axes for correct format so model can predict.
    arr = np.moveaxis(arr, 0, -1) #move axis to channels last
    new_arr = arr.reshape(-1, arr.shape[-1]) #reshape to row and column
    num_pixels = width*height
    new_arr2 = new_arr.reshape(num_pixels, 9, 7)
    print(new_arr2.shape)

    # Predict model and reshape to export.
    p = model.predict(new_arr2) # p is prediction from the DL model
    pim = p.reshape(width, height, 10) # Dimension of prediction in rows, columns, bands (10 classes)
    pim2 = np.moveaxis(pim, 2, 0) # move axis so bands is first
    
    # ArgMax for Segmentation.
    pim3 = np.argmax(pim2, axis=0) # take softmax of predictions for segmentation
    print(pim3.shape)

    # Get the file name (landsat_image_170_t.tif) by splitting input path.
    fileout_string = os.path.split(tif_image_list[t])
        
    # Output prediction raster .
    out_meta = ds.meta.copy()

    # Get Output metadata.
    out_meta.update({'driver':'GTiff',
                     'width':ds.shape[1],
                     'height':ds.shape[0],
                     'count':1,
                     'dtype':'float64',
                     'crs':ds.crs, 
                     'transform':ds.transform,
                     'nodata':0})

    # Write predicted raster to file.
    with rasterio.open(fp=outpath + "/prediction_" + fileout_string[-1], #outputpath_name
                 mode='w',**out_meta) as dst:
                 dst.write(pim3, 1) # the numer one is the number of bands
            
    print("Writing file...")
    


In [ ]:

import rasterio
from rasterio.plot import show
import glob

# Not used, but might be needed to index across tiles
tile_index = 170

# Input prediction .tif path
image_path = r'/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/datasets_oct22/rasters_gcloud/'

# Output prediction file path
outpath = r'/panfs/roc/groups/7/moeller/shared/leafy-spurge-demography/datasets_oct22/raster_predictions_gcloud/'

# List all .tif files in /rasters folder for prediction
tif_image_list = glob.glob(image_path + '*.tif')

print(tif_image_list[0:2])

# Loop through every tif file for prediction.
for t in range(len(tif_image_list)):
    
    # Open .tif array image with rasterio, read to numpy array
    with rasterio.open(tif_image_list[t], 'r') as ds:
        arr = ds.read()  # read all raster values

    # Define shape of input .tif image
    bands, width, height = arr.shape
    
    



In [ ]:

print(arr.shape)

print(arr.dtype)

